In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

## Income

In [ ]:
def plot_pay(year):
    """
    Year goes from 2002 to 2022 (incl)
    """
    # Validate the input year
    if year < 2002 or year > 2022:
        raise ValueError("Year must be between 2010 and 2022")

    # Load the shapefile
    shapefile_path = "London-wards-2018/London-wards-2018_ESRI/London_Ward.shp"
    gdf = gpd.read_file(shapefile_path)

    # Dissolve the geometries by the borough column to aggregate wards into boroughs
    boroughs = gdf.dissolve(by='DISTRICT').reset_index()

    # Load the CSV data
    csv_path = "pay.csv"
    csv_data = pd.read_csv(csv_path)

    # Extract relevant columns for the specified year
    year_str = str(year)
    pay_data = csv_data[['Area', year_str]]

    # Replace non-numeric values ('#') with NaN and convert to float
    pay_data[year_str] = pd.to_numeric(pay_data[year_str].replace('#', np.nan), errors='coerce')

    # Set the 'Area' column as the index
    pay_data = pay_data.set_index('Area')

    # Rename the column for consistency
    pay_data.columns = ['Pay']

    # Adjust the name in the CSV data to match 'City of Westminster'
    pay_data = pay_data.rename(index={'Westminster': 'City of Westminster'})

    # Merge the pay data with the borough geometries using 'DISTRICT' and 'Area'
    boroughs = boroughs.merge(pay_data, left_on='DISTRICT', right_index=True, how='left')

    # Check for any boroughs with missing data
    missing_data_boroughs = boroughs[boroughs['Pay'].isnull()]
    print("Boroughs with missing pay data:\n", missing_data_boroughs['DISTRICT'].tolist())

    # Plot the borders of the boroughs and fill with pay data for the specified year
    fig, ax = plt.subplots(figsize=(10, 10))
    boroughs.boundary.plot(ax=ax, color='black')
    boroughs.plot(column='Pay', cmap='Blues', legend=True, ax=ax, missing_kwds={"color": "red", "label": "Missing data"})

    # Remove axes and extra edges
    ax.set_axis_off()
    ax.margins(0)
    
    # Add legend for missing data
    handles, labels = ax.get_legend_handles_labels()
    handles.append(plt.Line2D([0], [0], marker='o', color='w', label='Missing data', markerfacecolor='red', markersize=10))
    ax.legend(handles=handles, loc='upper right')

    plt.title(f'Median Weekly Work-Based Earnings per person in {year} (in £)')
    plt.show()

## Trust 

In [ ]:

def plot_trust(year):
    """
    Trust data goes from 2016 to 2023 (incl)
    """
    # Validate the input year
    if year < 2016 or year > 2023:
        raise ValueError("Year must be between 2016 and 2023")

    # Load the shapefile
    shapefile_path = "London-wards-2018/London-wards-2018_ESRI/London_Ward.shp"
    gdf = gpd.read_file(shapefile_path)

    # Dissolve the geometries by the borough column to aggregate wards into boroughs
    boroughs = gdf.dissolve(by='DISTRICT').reset_index()

    # Load the CSV data for trust in MPS
    csv_path = "pas_data_borough (1).csv"
    csv_data = pd.read_csv(csv_path)

    # Filter the data for 'Trust MPS' measure and the specified year
    trust_mps_data = csv_data[(csv_data['Measure'] == 'Trust MPS') & (csv_data['Date'].str.contains(str(year)))]

    # Extract relevant columns
    trust_data = trust_mps_data[['Borough', 'Proportion']].copy()

    # Replace non-numeric values ('#') with NaN and convert to float
    trust_data.loc[:, 'Proportion'] = pd.to_numeric(trust_data['Proportion'].replace('#', np.nan), errors='coerce')

    # Set the 'Borough' column as the index
    trust_data = trust_data.set_index('Borough')

    # Rename the column for consistency
    trust_data.columns = ['Trust']

    # Adjust the name in the CSV data to match 'City of Westminster'
    trust_data = trust_data.rename(index={'Westminster': 'City of Westminster'})

    # Merge the trust data with the borough geometries using 'DISTRICT' and 'Borough'
    boroughs = boroughs.merge(trust_data, left_on='DISTRICT', right_index=True, how='left')

    # Check for any boroughs with missing data
    missing_data_boroughs = boroughs[boroughs['Trust'].isnull()]
    #print("Boroughs with missing trust data:\n", missing_data_boroughs['DISTRICT'].tolist())

    # Plot the borders of the boroughs and fill with trust data for the specified year
    fig, ax = plt.subplots(figsize=(10, 10))
    boroughs.boundary.plot(ax=ax, color='black')
    boroughs.plot(column='Trust', cmap='Blues', legend=True, ax=ax, 
                  missing_kwds={"color": "red", "label": "Missing data"})

    # Remove axes and extra edges
    ax.set_axis_off()
    ax.margins(0)

    # Add legend for missing data
    from matplotlib.patches import Patch
    handles, labels = ax.get_legend_handles_labels()
    handles.append(Patch(color='red', label='Missing data'))
    ax.legend(handles=handles, loc='upper right')

    # Set the title
    plt.title(f'Trust in MPS Proportion in {year}', pad=20)

    # Display the plot
    plt.show()

In [ ]:
plot_trust(2022)

In [ ]:
plot_pay(2022)

### Correlation calc

In [ ]:
def compute_correlation_trust_pay(year):
    # Validate the input year
    if year < 2016 or year > 2022:
        raise ValueError("Year must be between 2016 and 2022")

    # Load the CSV data for trust in MPS
    trust_csv_path = "pas_data_borough (1).csv"
    trust_data = pd.read_csv(trust_csv_path)

    # Filter the data for 'Trust MPS' measure and the specified year
    trust_mps_data = trust_data[(trust_data['Measure'] == 'Trust MPS') & (trust_data['Date'].str.contains(str(year)))]

    # Extract relevant columns and clean the data
    trust_mps_data = trust_mps_data[['Borough', 'Proportion']].copy()
    trust_mps_data['Proportion'] = pd.to_numeric(trust_mps_data['Proportion'].replace('#', np.nan), errors='coerce')
    trust_mps_data = trust_mps_data.rename(index={'Westminster': 'City of Westminster'})
    trust_mps_data.set_index('Borough', inplace=True)
    trust_mps_data.columns = ['Trust']

    # Load the CSV data for pay
    pay_csv_path = "pay.csv"
    pay_data = pd.read_csv(pay_csv_path)

    # Extract relevant columns for the specified year and clean the data
    pay_data = pay_data[['Area', str(year)]].copy()
    pay_data[str(year)] = pd.to_numeric(pay_data[str(year)].replace('#', np.nan), errors='coerce')
    pay_data = pay_data.rename(index={'Westminster': 'City of Westminster'})
    pay_data.set_index('Area', inplace=True)
    pay_data.columns = ['Pay']

    # Merge the datasets on the borough names
    merged_data = trust_mps_data.join(pay_data, how='inner')

    # Compute the correlation between trust and pay
    correlation = merged_data['Trust'].corr(merged_data['Pay'])

#     print(f"Correlation between trust in MPS and pay in {year}: {correlation}")

    return correlation

In [ ]:
years = list(range(2016, 2023))

# Compute correlations and store in a dictionary
correlations = {}
for year in years:
    correlations[year] = compute_correlation_trust_pay(year)

# Convert dictionary to DataFrame
correlation_df = pd.DataFrame(list(correlations.items()), columns=['Year', 'Correlation'])

# Display the results in a table
correlation_df
